# Multivariate Linear Regression

- Simple Linear Regression 복습
- Multiavriate Linear Regression 이론
- Naive Data Representation
- Matrix Data Representation
- Multivariate Linear Regression
- nn.Module 소개
- F.mse_loss

지금까지 Simple Linear Regression 은 하나의 정보로 부터 하나의 결론을 짖는 모델이였습니다. 

하지만, 다양한 문제를 풀기 위해선 이러한 하나의 정보론 결과를 얻을수가 없습니다.  
그렇기에 이제부터 Multiariate Linear Regression 에 대해 알아보도록 하겠습니다.  

## Multivariate Linear Regression

복수의 정보가 들어왔을때 하나의 정보를 추측하기  
ex) 3번의 쪽지시험 결과를 통해 기말고사 예측하기 

| quiz1 | quiz2 | quiz3 | Final(y) |
|-------|-------|-------|----------|
| 73    | 80    | 75    | 152      |
| 93    | 88    | 93    | 185      |
| 89    | 91    | 80    | 180      |
| 73    | 66    | 70    | 142       |

In [1]:
import torch

In [2]:
x_train = torch.FloatTensor([[73,80,75],[93,88,93],[89,91,80],[73,66,70]])
y_train = torch.FloatTensor([[152],[185],[180],[142]])

### Hypothesis Function

H(x) = Wx + b
- x라는 vector와 W라는 matrix의 곱

H(X) = w1x1 + w2x2 + w3x3 + b

위와 같이 x의 값이 적을떄는 hypothesis를 그냥 곱으로 구할 수 있지만 100개 1000개 10000개가 넘어가면 힘들다 .  
그럴떈 matmul()로 한번에 계산한다.  

matmul() 특징
- 간결하다.
- x의 길이가 바껴도 코드를 바꿀 필요가 없다
- 속도가 빠르다

In [3]:
# hypothesis = x_train.matmul(W) + b # or .mm or @ 를 사용

Multivariate Linear Regression 또한 Cost function을 구하는 공식은 기존 Simple Linear Regression과 동일하다.  

cost = torch.mean((hypothesis - y_train) ** 2)

# Full Code with torch.optim(1)

In [10]:
# 데이터
x_train = torch.FloatTensor([[73,80,75],
                             [93,88,93],
                             [89,91,80],
                             [73,66,70]])
y_train = torch.FloatTensor([[152],[185],[180],[142]])

# 모델의 초기화 (기존과 W가 바뀜)
W = torch.zeros((3,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정 (기존과 [W,b] 이 변경됌)
optimizer = torch.optim.SGD([W,b], lr=1e-5)

nb_epochs = 20

for epoch in range(nb_epochs+1) :
    hypothesis = x_train.matmul(W) + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    print('Epoch {:4d}/{} hypothesis: {} cost : {:6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0.]) cost : 27473.250000
Epoch    1/20 hypothesis: tensor([61.4952, 73.8967, 70.1744, 56.3782]) cost : 9981.958984
Epoch    2/20 hypothesis: tensor([ 98.5568, 118.4321, 112.4670,  90.3558]) cost : 3628.820801
Epoch    3/20 hypothesis: tensor([120.8928, 145.2724, 137.9560, 110.8333]) cost : 1321.251099
Epoch    4/20 hypothesis: tensor([134.3540, 161.4482, 153.3179, 123.1745]) cost : 483.100830
Epoch    5/20 hypothesis: tensor([142.4667, 171.1967, 162.5765, 130.6121]) cost : 178.669479
Epoch    6/20 hypothesis: tensor([147.3559, 177.0718, 168.1566, 135.0946]) cost : 68.093681
Epoch    7/20 hypothesis: tensor([150.3024, 180.6123, 171.5200, 137.7960]) cost : 27.929579
Epoch    8/20 hypothesis: tensor([152.0781, 182.7460, 173.5473, 139.4240]) cost : 13.339922
Epoch    9/20 hypothesis: tensor([153.1482, 184.0317, 174.7694, 140.4052]) cost : 8.039693
Epoch   10/20 hypothesis: tensor([153.7930, 184.8064, 175.5062, 140.9964]) cost : 6.113406
Epoch  

In [8]:
cost
W

tensor([[0.6895],
        [0.6825],
        [0.6643]], requires_grad=True)

위와 같이 구하는 방법도 있지만, 계속해서 W와 b를 구하는건 귀찮은 일입니다.  
pytorch는 이런점을 해결하기위해 nn.Module를 제공합니다.  

### nn.Module

- nn.Module 을 상속해서 모델 생성
- nn.Linear(3, 1)
    - 입력 차원 : 3
    - 출력 차원 : 1
- Hypothesis 계산은 foward()에서 !
- Gradient 계산은 Pytorch가 알아서 해준다 backward()

In [ ]:
import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.linear = nn.linear(3,1)
        
    def forward(self, x) :
        return self.linear(x)
    
hypothesis = model(x_train)

### torch.nn.functional

##### F.mse_loss

In [ ]:
# cost 계산
cost = cost = torch.mean((hypothesis - y_train) ** 2)

를 torch.nn.functional 모듈을 이용하면 편하게 계산 가능하다.  

이렇게 이용하면 가독성과 유지보수가 편하다.  
또한 다른 loss_function들로 교체가 간단하다. 

In [ ]:
import torch.nn.functional as F
# cost 계산

cost = F.mse_loss(prediction, y_train)

### Full Code with torch.optim(2)

In [ ]:
# H(x) 계산
# hypothesis = x_train.matmul(W) + b
Hypothesis = model(x_train)


# cost 계산
# cost = torch.mean((hypothesis - y_train) ** 2)
cost = F.mse_loss(prediction, y_train)

들로 변경한다.